https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b
https://www.kaggle.com/code/waxbabi/light-gbm-benchmark-0-3692
https://asagar60.medium.com/instacart-market-basket-analysis-part-1-introduction-eda-b08fd8250502

In [2]:
import os
import numpy as np
import pandas as pd

import gc # 이게 뭔지는 모르지만 일단 기다려보자.

# Problem Overview

The main objective of this competition was to predict which previously purchased products will be in the user's next order.

we are not recommending new products to the user instead of that we are going to recommend the product which is previously bought by that user.

### 과거에 산 상품들 중에서, 유저가 재구매할 상품을 추천하는 문제

At first, it seems like Multi-Label Classification, but there are 49688 products, and total product recommendations could be anywhere from None to N. Therefore, this problem is restructured into a binary classification problem, where we will predict the probability of an item being reordered by a user.

For each order, we will group these probabilities to  pick top K probable products which will be reordered, and recommend those to the user.

1. 만약 어떤 유저 A가 90번의 주문을 했다고 가정, 이는 prior_orders_data에 저장되어 있음
2. 89번까지의 주문을 사용해 features engineering을 진행
3. 89에서 주문한 상품 중, 90번에서 또 주문할 확률을 구해 상품을 추천

### Multi-Label Classification이 아닌 Binary Classification으로 진행. order별로 구매할 확률이 높은 K개의 상품을 뽑고 해당 상품이 재주문 되었는지로 평가

재구매 여부와 동시에 추천여부를 결정하는 과정에서 미리 설정한 Threshold 값을 사용함
- Threshold의 값을 GridSearchCV를 활용한 결과, 0.5가 아닌 0.22로 진행함

## Model 종류
* Reorder Prediction : probability of a product being redordered by user in next order (6개의 model, Average)
* None Prediction : probability of a reordered product to be in user's next order (17개의 model, Weighted Average)

## F1 Score Maximization
지금 이 부분에서 이해가 안됨
* F1 Score를 최대화하는 threshold를 만듬
* 그런데 여기서 A, B 상품이 무엇을 의미하는지 모르겠음
* Let, reorder probability of product, P(A) = 0.9 and P(B) = 0.3
* F1(A) > F1(A B) 이 부분을 보니 뭔가 F1 Score가 평가 Metric이 아닌 출력관련 내용같음

aisle은 대분류로 생각하면 될거같음

# Data Overview

In [3]:
base_path = '../Data/InstaCart'

In [4]:
priors = pd.read_csv(os.path.join(base_path, 'order_products__prior.csv'))
train = pd.read_csv(os.path.join(base_path, 'order_products__train.csv'))
orders = pd.read_csv(os.path.join(base_path, 'orders.csv'))
products = pd.read_csv(os.path.join(base_path, 'products.csv'))

* order_id : 주문 고유 아이디
* user_id : 소비자 고유 아이디
* product_id : 제품 고유 아이디
* eval_set : 소비자 군(prior / train / test)
* order_number : 주문번호
* order_dow : 주문 요일
* order_hoour_of_day : 일일 중 시간
* days_since_prior_order : 
* add_to_cart_order : 장바구니 담은 제품 개수
* reodered : 재주문 제품 개수
* product_name : 제품 이름
* aisle_id : 제품 상세카테고리 고유 아이디
* department_id : 제품 카테고리 고유 아이디
* aisle : 제품 상세카테고리
* department : 제품 카테고리

### 주문정보

This table includes all orders, namely prior, train and test. It has a single primary key (order_id)

In [52]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [53]:
orders.order_dow = orders.order_dow.astype(np.int8)
orders.order_hour_of_day = orders.order_hour_of_day.astype(np.int8)
orders.order_number = orders.order_number.astype(np.int16)
orders.order_id = orders.order_id.astype(np.int32)
orders.user_id = orders.user_id.astype(np.int32)
orders.days_since_prior_order = orders.days_since_prior_order.astype(np.float32)

### (과거) 구매자군의 제품주문내역
Order history of every user. This data contains nearly 3-100 past orders per user

In [48]:
priors.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [49]:
priors.order_id = priors.order_id.astype(np.int32)
priors.add_to_cart_order = priors.add_to_cart_order.astype(np.int16)
priors.reordered = priors.reordered.astype(np.int8)
priors.product_id = priors.product_id.astype(np.int32)

### (현재) 구매자군의 제품주문내역

Current order data of every user. This data contains only 1 order per user

In [50]:
train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [51]:
train.reordered = train.reordered.astype(np.int8)
train.add_to_cart_order = train.add_to_cart_order.astype(np.int16)

### 제품정보

In [54]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [55]:
products.drop(['product_name'], axis=1, inplace=True)
products.aisle_id = products.aisle_id.astype(np.int8)
products.department_id = products.department_id.astype(np.int8)
products.product_id = products.product_id.astype(np.int32)

In [56]:
print('computing product f')
prods = pd.DataFrame()
prods['orders'] = priors.groupby(priors.product_id).size().astype(np.float32)
prods['reorders'] = priors['reordered'].groupby(priors.product_id).sum().astype(np.float32)
prods['reorder_rate'] = (prods.reorders / prods.orders).astype(np.float32)
products = products.join(prods, on = 'product_id')
del prods

computing product f


In [57]:
print('add order info to priors')
orders.set_index('order_id', inplace = True, drop = False)
priors = priors.join(orders, on = 'order_id', rsuffix = '_')
priors.drop('order_id_', inplace = True, axis = 1)

add order info to priors


In [60]:
print('cumputing user f')
usr = pd.DataFrame()
usr['average_days_between_orders'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)
usr['nb_orders'] = orders.groupby('user_id').size().astype(np.int16)

users = pd.DataFrame()
users['total_items'] = priors.groupby('user_id').size().astype(np.int16)
users['all_products'] = priors.groupby('user_id')['product_id'].apply(set)
users['total_distinct_items'] = (users.all_products.map(len)).astype(np.int16)

users = users.join(usr)
del usr
users['average_basket'] = (users.total_items / users.nb_orders).astype(np.float32)
gc.collect()
print('user f', users.shape)

cumputing user f
user f (206209, 6)


In [61]:
priors['user_product'] = priors.product_id+priors.user_id * 100000

In [ ]:
d = dict()
for row in priors.itertyples():
    z = row.user_product
    
    if z not in d:
        d[z] = (1, (row.order_number, row.order_id), row.add_to_cart_order)
    else:
        